# Precomupte forward model results

In [ ]:
import pyvista as pv
pv.set_jupyter_backend('static')

In [ ]:
from functools import lru_cache
import os

import numpy as np
import xarray as xr

import cedalion.datasets
import cedalion.geometry.segmentation
import cedalion.imagereco.forward_model as fw
import cedalion.io.forward_model
import cedalion.plots
xr.set_options(display_expand_data=False);

In [ ]:
def compute_fluence_mcx(rec, head):
    geo3d_snapped_ijk = head.align_and_snap_to_scalp(rec.geo3d)

    fwm = cedalion.imagereco.forward_model.ForwardModel(
        head, geo3d_snapped_ijk, rec._measurement_lists["amp"]
    )

    fluence_all, fluence_at_optodes = fwm.compute_fluence_mcx()

    return fluence_all, fluence_at_optodes


def plot_fluence(rec, head, fluence_all, src, det, wl):
    geo3d_snapped_ijk = head.align_and_snap_to_scalp(rec.geo3d)

    f = fluence_all.loc[src, wl].values * fluence_all.loc[det, wl].values

    # clip fluence to smallest positive value and transform to log
    f[f <= 0] = f[f > 0].min()
    f = np.log10(f)

    vf = pv.wrap(f)

    plt = pv.Plotter()
    plt.add_volume(
        vf,
        log_scale=False,
        cmap="plasma_r",
        clim=(-10, 0),
    )
    cedalion.plots.plot_surface(plt, head.brain, color="w")
    cedalion.plots.plot_labeled_points(plt, geo3d_snapped_ijk, show_labels=False)

    cog = head.brain.vertices.mean("label").values
    plt.camera.position = cog + [-300, 30, 150]
    plt.camera.focal_point = cog
    plt.camera.up = [0, 0, 1]

    plt.show()


@lru_cache
def get_colin27():
    SEG_DATADIR, mask_files, landmarks_file = (
        cedalion.datasets.get_colin27_segmentation()
    )

    head = fw.TwoSurfaceHeadModel.from_segmentation(
        segmentation_dir=SEG_DATADIR,
        mask_files=mask_files,
        landmarks_ras_file=landmarks_file,
    )

    return head


@lru_cache
def get_icbm152():
    SEG_DATADIR, mask_files, landmarks_file = (
        cedalion.datasets.get_icbm152_segmentation()
    )

    head = fw.TwoSurfaceHeadModel.from_surfaces(
        segmentation_dir=SEG_DATADIR,
        mask_files=mask_files,
        brain_surface_file=os.path.join(SEG_DATADIR, "mask_brain.obj"),
        landmarks_ras_file=landmarks_file,
        brain_face_count=None,
        scalp_face_count=None,
    )

    return head


# fluence_all, fluence_at_optodes = plot_fluence(rec, head, fluence_all, "S4", "D2", 760.)

In [ ]:
rec = cedalion.datasets.get_fingertappingDOT()
head = get_colin27()
fluence_all, fluence_at_optodes = compute_fluence_mcx(rec, head)

cedalion.io.forward_model.save_fluence("fluence_fingertappingDOT_colin27.h5", fluence_all, fluence_at_optodes)

In [ ]:
rec = cedalion.datasets.get_fingertappingDOT()
head = get_icbm152()
fluence_all, fluence_at_optodes = compute_fluence_mcx(rec, head)
cedalion.io.forward_model.save_fluence("fluence_fingertappingDOT_icbm152.h5", fluence_all, fluence_at_optodes)

In [ ]:
rec = cedalion.datasets.get_fingertapping()
head = get_colin27()
fluence_all, fluence_at_optodes = compute_fluence_mcx(rec, head)
cedalion.io.forward_model.save_fluence("fluence_fingertapping_colin27.h5", fluence_all, fluence_at_optodes)

In [ ]:
rec = cedalion.datasets.get_fingertapping()
head = get_colin27()
fluence_all, fluence_at_optodes = compute_fluence_mcx(rec, head)
cedalion.io.forward_model.save_fluence("fluence_fingertapping_icbm152.h5", fluence_all, fluence_at_optodes)